In [1]:
from os import listdir
from os.path import isfile, join, normpath
from os import walk, rename
import random

In [2]:
import numpy as np
import cv2 # OpenCV
import matplotlib
import matplotlib.pyplot as plt
import collections

In [3]:
# keras
from keras.models import Sequential, load_model
from keras.layers import Conv2D, Activation, Dropout, MaxPooling2D, Flatten, Dense, \
    LeakyReLU, BatchNormalization, AveragePooling2D
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.losses import categorical_crossentropy
from keras import regularizers
from keras.callbacks import TensorBoard

Using TensorFlow backend.


In [4]:
TRAIN_IMAGES = []
TRAIN_LABELS = []

In [5]:
IMG_ROWS = 48
IMG_COLS = 48
DATASET_PATH = "processed_dataset/"
CATEGORIES = ["BOOK", "FILM", "SHOW", "GEO", "PHRASE", "SERIES", "ONE", "TWO", "THREE", "FOUR", "FIVE", "SIX", "SEVEN", 
              "EIGHT", "NINE", "TEN" ]
BATCH_SIZE = 32
EPOCHS = 15

In [6]:
def create_label(img_name):
    #categories
    if "book" in img_name:
        return 0
    if "film" in img_name:
        return 1
    if "show" in img_name:
        return 2
    if "geo" in img_name:
        return 3
    if "phrase" in img_name:
        return 4
    if "series" in img_name:
        return 5
    #numbers
    if "one" in img_name:
        return 6
    if "two" in img_name:
        return 7
    if "three" in img_name:
        return 8
    if "four" in img_name:
        return 9
    if "five" in img_name:
        return 10
    if "six" in img_name:
        return 11
    if "seven" in img_name:
        return 12
    if "eight" in img_name:
        return 13
    if "nine" in img_name:
        return 14
    if "ten" in img_name:
        return 15
    
    

In [7]:
def load_dataset(): 
    all_images = []
    loaded_images = [f for f in listdir(DATASET_PATH) if isfile(join(DATASET_PATH, f))]
    for img_name in loaded_images:
        image = cv2.imread(DATASET_PATH + img_name,  cv2.IMREAD_GRAYSCALE)
        image = image/255.0
        image = cv2.resize(image, (IMG_ROWS, IMG_COLS))
        image = image.flatten()
        label = create_label(img_name)
        all_images.append([image, label])
    # shuffle the data
    random.shuffle(all_images)
    
    # create np arrays
    train_x = []
    train_y = []

    for features, label in all_images:
        train_x.append(features)
        train_y.append(label)

    # reshape np array
    train_x = np.array(train_x).reshape(-1, IMG_ROWS, IMG_COLS, 1)
    train_y = np.array(train_y)
    train_y = to_categorical(train_y, 16)
    return train_x, train_y

In [8]:
def create_model():
    model = Sequential()

    model.add(Conv2D(48, padding='same', kernel_size=(3, 3), input_shape=(IMG_ROWS, IMG_COLS, 1)))
    model.add(Activation("relu"))
    model.add(Dropout(0.2))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(48, padding='same', kernel_size=(3, 3)))
    model.add(Activation("relu"))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.3))

    model.add(Conv2D(48, padding='same', kernel_size=(3, 3)))
    model.add(Activation("relu"))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.4))

    model.add(Flatten())

    model.add(Dense(16, kernel_regularizer=regularizers.l1(0.001)))
    model.add(Activation("softmax"))

    model.summary()
    return model

In [9]:
def compile_fit_model(model, train_x, train_y):
    print(len(train_x))
    print(len(train_y))
    # Compile model
    model.compile(loss=categorical_crossentropy,
                  optimizer=Adam(),
                  metrics=["accuracy"])

    # Train and evaluation
    model.fit(train_x, train_y,
              batch_size=BATCH_SIZE, epochs=EPOCHS,
              validation_split=0.2, verbose=1)
    model.save('Charades-CNN-48x3-ThreeDropout-Batch-32-l1-0x001' + '.model')

In [10]:
#load dataset
TRAIN_IMAGES, TRAIN_LABELS = load_dataset()

In [11]:
# create CNN model
cnn_model = create_model()
compile_fit_model(cnn_model, TRAIN_IMAGES, TRAIN_LABELS)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 48, 48, 48)        480       
_________________________________________________________________
activation_1 (Activation)    (None, 48, 48, 48)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 48, 48, 48)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 24, 24, 48)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 24, 24, 48)        20784     
_________________________________________________________________
activation_2 (Activation)    (None, 24, 24, 48)        0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 12, 12, 48)       

In [ ]:
#load saved model
saved_model = load_model('Charades-CNN-48x3-ThreeDropout-Batch-32-l1-0x001' + '.model')